# Tutorial: Actor Critic Implementation

In [1]:
#Import required libraries

import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
#Set constants for training
seed = 543
log_interval = 10
gamma = 0.99

env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)

SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)


SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values


In [4]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [5]:

def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]


def train():
    running_reward = 10

    # run infinitely many episodes
    for i_episode in range(2000):

        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _ = env.step(action)

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
train()

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 0	Last reward: 22.00	Average reward: 10.60
Episode 10	Last reward: 28.00	Average reward: 16.78
Episode 20	Last reward: 42.00	Average reward: 33.66
Episode 30	Last reward: 21.00	Average reward: 31.73
Episode 40	Last reward: 26.00	Average reward: 29.00
Episode 50	Last reward: 150.00	Average reward: 64.74
Episode 60	Last reward: 85.00	Average reward: 87.97
Episode 70	Last reward: 234.00	Average reward: 153.18
Episode 80	Last reward: 44.00	Average reward: 144.46
Episode 90	Last reward: 44.00	Average reward: 101.80
Episode 100	Last reward: 133.00	Average reward: 89.91
Episode 110	Last reward: 60.00	Average reward: 88.15
Episode 120	Last reward: 339.00	Average reward: 178.64
Episode 130	Last reward: 105.00	Average reward: 177.38
Episode 140	Last reward: 34.00	Average reward: 138.97
Episode 150	Last reward: 500.00	Average reward: 155.96
Episode 160	Last reward: 500.00	Average reward: 267.45
Episode 170	Last reward: 500.00	Average reward: 360.77
Episode 180	Last reward: 140.00	Average 

# TODO: Write a policy class similar to the above, without using shared features for the actor and critic and compare their performance.

In [7]:

class UnsharedPolicy(nn.Module):
  def __init__(self):
    super(UnsharedPolicy, self).__init__()

    # actor's features
    self.affine1 = nn.Linear(4, 128)

    # critic's features
    self.affine2 = nn.Linear(4, 128)

    # actor's layer
    self.action_head = nn.Linear(128, 2)

    # critic's layer
    self.value_head = nn.Linear(128, 1)

    # action & reward buffer
    self.saved_actions = []
    self.rewards = []


  def forward(self, x):

      # actor: choses action to take from state s_t
      # by returning probability of each action
      x1 = F.relu(self.affine1(x))
      action_prob = F.softmax(self.action_head(x1), dim=-1)

      # critic: evaluates being in the state s_t
      x2 = F.relu(self.affine2(x))
      state_values = self.value_head(x2)

      return action_prob, state_values

In [8]:
model = UnsharedPolicy()
optimizer = optim.Adam(model.parameters(), lr=3e-3)   # changed learning rate to 3e-3 for better convergence
eps = np.finfo(np.float32).eps.item()
train()

Episode 0	Last reward: 15.00	Average reward: 10.25
Episode 10	Last reward: 37.00	Average reward: 17.28
Episode 20	Last reward: 54.00	Average reward: 27.97
Episode 30	Last reward: 41.00	Average reward: 34.51
Episode 40	Last reward: 47.00	Average reward: 43.38
Episode 50	Last reward: 55.00	Average reward: 43.32
Episode 60	Last reward: 47.00	Average reward: 53.15
Episode 70	Last reward: 46.00	Average reward: 58.14
Episode 80	Last reward: 161.00	Average reward: 76.27
Episode 90	Last reward: 75.00	Average reward: 82.74
Episode 100	Last reward: 132.00	Average reward: 104.46
Episode 110	Last reward: 165.00	Average reward: 105.53
Episode 120	Last reward: 196.00	Average reward: 127.27
Episode 130	Last reward: 93.00	Average reward: 165.32
Episode 140	Last reward: 169.00	Average reward: 187.22
Episode 150	Last reward: 500.00	Average reward: 291.00
Episode 160	Last reward: 476.00	Average reward: 331.94
Episode 170	Last reward: 214.00	Average reward: 345.41
Episode 180	Last reward: 171.00	Average r

In [10]:
for lr in [3e-2, 3e-3, 3e-4]:
  print("*" * 60)
  print(f"Learning rate = {lr}")
  model = UnsharedPolicy()
  optimizer = optim.Adam(model.parameters(), lr=lr)
  eps = np.finfo(np.float32).eps.item()
  train()

************************************************************
Learning rate = 0.03
Episode 0	Last reward: 27.00	Average reward: 10.85
Episode 10	Last reward: 9.00	Average reward: 10.15
Episode 20	Last reward: 9.00	Average reward: 9.93
Episode 30	Last reward: 10.00	Average reward: 9.84
Episode 40	Last reward: 9.00	Average reward: 9.59
Episode 50	Last reward: 10.00	Average reward: 9.61
Episode 60	Last reward: 10.00	Average reward: 9.52
Episode 70	Last reward: 8.00	Average reward: 9.32
Episode 80	Last reward: 11.00	Average reward: 9.38
Episode 90	Last reward: 9.00	Average reward: 9.37
Episode 100	Last reward: 8.00	Average reward: 9.43
Episode 110	Last reward: 9.00	Average reward: 9.42
Episode 120	Last reward: 10.00	Average reward: 9.35
Episode 130	Last reward: 10.00	Average reward: 9.40
Episode 140	Last reward: 10.00	Average reward: 9.53
Episode 150	Last reward: 10.00	Average reward: 9.61
Episode 160	Last reward: 8.00	Average reward: 9.45
Episode 170	Last reward: 10.00	Average reward: 9.42

In [11]:
for lr in [9e-3, 3e-3, 1e-3]:
  print("*" * 60)
  print(f"Learning rate = {lr}")
  model = UnsharedPolicy()
  optimizer = optim.Adam(model.parameters(), lr=lr)
  eps = np.finfo(np.float32).eps.item()
  train()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


************************************************************
Learning rate = 0.009
Episode 0	Last reward: 12.00	Average reward: 10.10
Episode 10	Last reward: 15.00	Average reward: 16.11
Episode 20	Last reward: 13.00	Average reward: 14.62
Episode 30	Last reward: 13.00	Average reward: 13.11
Episode 40	Last reward: 9.00	Average reward: 12.01
Episode 50	Last reward: 12.00	Average reward: 11.79
Episode 60	Last reward: 11.00	Average reward: 11.86
Episode 70	Last reward: 13.00	Average reward: 14.17
Episode 80	Last reward: 52.00	Average reward: 25.68
Episode 90	Last reward: 19.00	Average reward: 24.02
Episode 100	Last reward: 58.00	Average reward: 35.81
Episode 110	Last reward: 71.00	Average reward: 43.27
Episode 120	Last reward: 46.00	Average reward: 54.66
Episode 130	Last reward: 57.00	Average reward: 69.55
Episode 140	Last reward: 135.00	Average reward: 83.69
Episode 150	Last reward: 177.00	Average reward: 104.62
Episode 160	Last reward: 63.00	Average reward: 98.59
Episode 170	Last reward: 

In [14]:
import time

start_time = time.time()

model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()
train()

end_time = time.time()

print("Shared policy execution time:", end_time - start_time, "seconds")

Episode 0	Last reward: 16.00	Average reward: 10.30
Episode 10	Last reward: 21.00	Average reward: 17.52
Episode 20	Last reward: 34.00	Average reward: 23.19
Episode 30	Last reward: 73.00	Average reward: 37.74
Episode 40	Last reward: 22.00	Average reward: 38.62
Episode 50	Last reward: 32.00	Average reward: 32.44
Episode 60	Last reward: 48.00	Average reward: 38.46
Episode 70	Last reward: 60.00	Average reward: 50.22
Episode 80	Last reward: 79.00	Average reward: 54.99
Episode 90	Last reward: 116.00	Average reward: 69.43
Episode 100	Last reward: 124.00	Average reward: 167.68
Episode 110	Last reward: 117.00	Average reward: 169.96
Episode 120	Last reward: 80.00	Average reward: 148.72
Episode 130	Last reward: 500.00	Average reward: 256.88
Episode 140	Last reward: 500.00	Average reward: 354.44
Episode 150	Last reward: 77.00	Average reward: 325.79
Episode 160	Last reward: 22.00	Average reward: 277.51
Episode 170	Last reward: 12.00	Average reward: 170.33
Episode 180	Last reward: 13.00	Average rewar

In [15]:
start_time = time.time()

model = UnsharedPolicy()
optimizer = optim.Adam(model.parameters(), lr=3e-3)
eps = np.finfo(np.float32).eps.item()
train()

end_time = time.time()

print("Unshared policy execution time:", end_time - start_time, "seconds")

Episode 0	Last reward: 26.00	Average reward: 10.80
Episode 10	Last reward: 124.00	Average reward: 21.80
Episode 20	Last reward: 15.00	Average reward: 28.63
Episode 30	Last reward: 30.00	Average reward: 35.63
Episode 40	Last reward: 24.00	Average reward: 41.94
Episode 50	Last reward: 55.00	Average reward: 54.43
Episode 60	Last reward: 71.00	Average reward: 64.87
Episode 70	Last reward: 68.00	Average reward: 76.62
Episode 80	Last reward: 130.00	Average reward: 88.05
Episode 90	Last reward: 153.00	Average reward: 106.83
Episode 100	Last reward: 175.00	Average reward: 147.85
Episode 110	Last reward: 147.00	Average reward: 182.63
Episode 120	Last reward: 185.00	Average reward: 189.48
Episode 130	Last reward: 228.00	Average reward: 203.60
Episode 140	Last reward: 389.00	Average reward: 247.77
Episode 150	Last reward: 104.00	Average reward: 209.91
Episode 160	Last reward: 123.00	Average reward: 173.89
Episode 170	Last reward: 160.00	Average reward: 159.92
Episode 180	Last reward: 206.00	Avera

## Observations

- Among different learning rates, `3e-3` seems the most appropriate learning rate for the unshared policy, in comparison to the default value of `3e-2` given in the problem statement.

- For this given task, both shared and unshared policies (sharing with reference to sharing features between actor and critic) are able to show convergence robust over multiple runs of the algorithm. For this task, shared policy seems to generally take lesser episodes (not always, as shown by the above run) to "solve" the problem when compared to the unshared policy.

- The per-episode runtime is similar for both policies, with the unshared one taking a little more time than the shared policy.